In [1]:
import numpy as np
import tensorflow as tf

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

import seaborn
import toyplot


np.random.seed(3)
tf.set_random_seed(3)
tf.__version__

'1.4.1'

jupyter nbconvert ownNLP.ipynb --to slides --post serve

<div><img src="presentationFiles/graph_tf.png" width=400 align="center"/></div>

# 'Differentiable Learning of Logical Rules for Knowledge base Reasoning' by Yang, Yang and Cohen
#### a presentation by Nico Lutz for KGA Seminar WS 1718 Universität Bonn

Testing an own simple implementation of NLP decsribed in ['Differentiable Learning of Logical Rules for Knowledge base Reasoning'](https://arxiv.org/abs/1702.08367) by Yang, Yang and Cohen. This code does nothing new. As a preparation for a presentation I decided to compare the paper to the source code. As a result heavy copy and pasting redoes what works best as the original code. This is a very step by step implementation to go through the formulas and see what happens to the matrices and is by no means comparable to [original nlp implementation](https://github.com/fanyangxyz/Neural-LP) subsequently this notebook was used as source for a presentation


### Table of Content
1. Primer: First order Logical Rules
2. Motivation
3. Tensorlog for KB Reasoning
4. Differentiable Learning part 1
5. Primer: Recurrent Neuronal Networks
6. Primer: Attention Mechanism
7. Differentiable Learning part 2 & Architecture
8. Recovery of Logical Rules
9. Experiments
10. Evaluation Metric
11. Experiment: query in natural language
12. Summary

# 1. Primer: First order Logical Rules

**Facts:** 

* $HasOfficeInCity(Uber ,NewYork)$. 
* $HasOfficeInCity(Lyft ,Paris)$.
* $CityInCountry(NewYork, USA)$. 
* $CityInCountry(Paris, France)$.

**Rules:**

\begin{equation*}
HasOfficeInCountry(Y, X) {\leftarrow} HasOfficeInCity(Y, Z) {\wedge} CityInCountry(Z,X)
\end{equation*}


**Query and Inference:** 

In which country X does Uber have an Office?
-> Inference: $HasOfficeInCountry(Uber, X)$. Answer: $X = USA$.

### Terminology

* Knowledge Base are collections of relational data of the form **Relation(Head, Tail)**. E.g.: $HasOfficeInCity(Lyft , Paris)$
* **Entities**: Head, Tail. E.g.: $Paris$, $Lyft$
* Relation is a binary Relation between Entities.

# 2. Motivation

#### Goal:

* Example: In the KB the fact $HasOfficeInCountry(Uber,USA)$ is missing. Reason over existing facts to infer $Uber$ when presented with query: $HasOfficeInCountry(Y,USA)$

* In other words: The goal is to retrive a ranked list of entites based on the query such that the desired answer is ranked as high as possible. The query is of the from query(head,tail), such that head is the answer to the query given tail.

So we are interested in learning weighted chain-like logical rules of the from:

\begin{equation*}
\mathbf{\alpha} \: query(Y,X) {\leftarrow} R_{n}(Y,Z_{n}) {\wedge} ... {\wedge}R_{1}(Z_{1},X) \tag{1}
\end{equation*}


where $\mathbf{\alpha} {\in} [0,1]$ is the confidence associated with that rule and $R_{1},...,R_{n}$ are relations in the knowledge base.

This means:

\begin{equation*}
0.9 * HasOfficeInCountry(Y, X) {\leftarrow} HasOfficeInCity(Y, Z) {\wedge} CityInCountry(Z,X)
\end{equation*}

# 3. Tensorlog for KB reasoning

### Entity
Each **entity** of the set of Entites $E$ is assigned an integer and each entity $i$ is assigned a one hot encoded vector $\mathbf{v_{i}} {\in} [0,1]^{E}$ such that the i-th entry is 1.

In [2]:
def one_hot(entities, num_entity):
    '''
    converts a list of entries to a one hot matrix
    '''
    E_one_hot = np.zeros((num_entity, num_entity))
    E_one_hot[np.arange(num_entity),entities] = 1
    return E_one_hot

In [3]:
def print_entities(matrix, name, label=['Uber/0', 'Lyft/1', 'USA/2', 'NewYork/3', 'France/4', 'Paris/5'],lrange=[0,1,2,3,4,5], domain_mi=0, domain_ma=1):
    colormap = toyplot.color.brewer.map("Paired",domain_min=domain_mi, domain_max=domain_ma)    
    tlocator = toyplot.locator.Explicit(lrange)
    llocator = toyplot.locator.Explicit(lrange, label)
    canvas, table = toyplot.matrix((matrix, colormap), tlocator=tlocator, llocator=llocator, label=name, width=350);
    table.body.gaps.columns[...] = 2
    table.body.gaps.rows[...] = 2

In [4]:
Uber = 0
Lyft = 1
USA = 2
NewYork = 3
France = 4
Paris = 5

Entities = [Uber, Lyft, USA, NewYork, France, Paris]
Entites_one_hot_encoded = one_hot(Entities, len(Entities))

print_entities(Entites_one_hot_encoded, name='Entites one hot encoded')

Entites one hot encoded 0 1 2 3 4 5 Uber/0 1.0 0.0 0.0 0.0 0.0 0.0 Lyft/1 0.0 1.0 0.0 0.0 0.0 0.0 USA/2 0.0 0.0 1.0 0.0 0.0 0.0 NewYork/3 0.0 0.0 0.0 1.0 0.0 0.0 France/4 0.0 0.0 0.0 0.0 1.0 0.0 Paris/5 0.0 0.0 0.0 0.0 0.0 1.0

### Relations

Each **relation** is defined as a Tensorlog operator $M_{R}$. So that each relation $M_{R}$ is matrix in $\{0,1\}^{ExE}$.

Example: A matrix $M_{HasOfficeInCity(Uber, NewYork)}$ looks like:

In [5]:
def make_matrix(num_entity, head_tail):
    '''
    builds a matrix for a relation based on its head tail
    
    return mr
    '''
    m = np.zeros((num_entity,num_entity))
    h = head_tail
    m[h[0],h[1]] = 1
    return m

In [6]:
def print_relation(matrix, name, domain_mi=0, domain_ma=1):
    colormap = toyplot.color.brewer.map("Paired",domain_min=domain_mi, domain_max=domain_ma)    
    canvas, table = toyplot.matrix((matrix, colormap), label=name, width=350);
    table.body.gaps.columns[...] = 2
    table.body.gaps.rows[...] = 2

In [7]:
HasOfficeInCity = 0
CityInCountry = 1
HasOfficeInCountry = 2
CountryIsInEurope = 3
CountryIsInAsia = 4

relations = [HasOfficeInCity, CityInCountry, HasOfficeInCountry, CountryIsInEurope, CountryIsInAsia]

In [8]:
M_HasOfficeInCity = make_matrix(len(Entities), [Uber, NewYork] )
print_relation(M_HasOfficeInCity, name='M_HasOfficeInCity')

M_HasOfficeInCity 0 1 2 3 4 5 0 0.0 0.0 0.0 1.0 0.0 0.0 1 0.0 0.0 0.0 0.0 0.0 0.0 2 0.0 0.0 0.0 0.0 0.0 0.0 3 0.0 0.0 0.0 0.0 0.0 0.0 4 0.0 0.0 0.0 0.0 0.0 0.0 5 0.0 0.0 0.0 0.0 0.0 0.0

### Inference
By performing matrix multiplication a statement like: $R(Y,X) {\leftarrow} P(Y,Z) {\wedge} Q(Z,X)$ for any entity $X  = x$ becomes: $M_{p}*M_{Q}*v_{x} = s$

Example: $HasOfficeInCountry(Y, X) {\leftarrow} HasOfficeInCity(Y, Z) {\wedge} CityInCountry(Z,X)$, $Z = NewYork, X = USA$

In [9]:
def print_solution(M_p, M_q, v_x, s):
    '''
    pretty prints the solution to M_p*M_q*v_x = s
    
    the old way
    '''
    fmt  = '{:<26}  {:<26}  {:<5}  {}'
    fmt2 = '{:<26}* {:<26}* {:<5}= {}'

    print(fmt.format('M_p', 'M_q','v_x','s'))
    for i, (q, p, v, s_) in enumerate(zip(M_p, M_q, v_x, s)):
        if i == 3:
            print(fmt2.format(q, p, v,s_))
        else:
            print(fmt.format(q, p, v,s_))

In [10]:
def print_inference(M_p, M_q, v_x, s):
    '''
    pretty prints the solution to M_p*M_q*v_x = s
    
    the new way
    '''
    colormap = toyplot.color.brewer.map("Paired",domain_min=1, domain_max=0)    
    canvas = toyplot.Canvas(width=900, height=300, style = {})
    ta = canvas.matrix((M_p,colormap), label="M_p", bounds=(0, 250, 30, 280), tshow=False, lshow=False)
    ta.body.gaps.columns[...] = 2
    ta.body.gaps.rows[...] = 2
    canvas.text(255, 165, "<b> * </b>",style={"font-size":"32px"})
    tab = canvas.matrix((M_q,colormap), label="M_q", bounds=(260, 510, 30, 280), tshow=False, lshow=False)
    tab.body.gaps.columns[...] = 2
    tab.body.gaps.rows[...] = 2
    canvas.text(515, 165, "<b> * </b>",style={"font-size":"32px"})
    tabl = canvas.matrix((np.transpose(np.asarray(np.asmatrix(v_x))),colormap), label="v_x", bounds=(520, 620, 30, 280), tshow=False, lshow=False)
    tabl.body.gaps.columns[...] = 2
    tabl.body.gaps.rows[...] = 2
    canvas.text(625, 155, "<b> = </b>",style={"font-size":"32px"})
    table= canvas.matrix((np.transpose(np.asarray(np.asmatrix(s))),colormap), label="s", bounds=(630, 730, 30, 280), tshow=False, lshow=False)
    table.body.gaps.columns[...] = 2
    table.body.gaps.rows[...] = 2

In [11]:
# HasOfficeInCity(Uber, NewYork)
M_p = make_matrix(len(Entities), [Uber, NewYork] )

# CityInCountry(NewYork, USA)
M_q = make_matrix(len(Entities), [NewYork, USA] )

# One hot encoding of USA
v_x = Entites_one_hot_encoded[2]

# inference = should be One hot encoding of Uber
s = np.matmul(np.matmul(M_p,M_q),v_x)

print_inference(M_p,M_q, v_x, s)

M_p 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 * M_q 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 * v_x 0.0 0.0 1.0 0.0 0.0 0.0 = s 1.0 0.0 0.0 0.0 0.0 0.0

# 4. Differentiable Learning part 1

Using the Tensorlog Operators we want to learn for each query:

\begin{equation*}
\sum_{l} {\alpha}_l \prod_{k \in {\beta}_l} M_{R_{k}} \tag{2} 
\end{equation*}

<small>where $l$ indexes all possible rules, ${\alpha}_l$ is the confidence associated with that rule $l$ and ${\beta}_l$ is an ordered list of all relations in this particular rule</small>

During inference given an entity $v_x$

\begin{equation*}
s = \sum_{l} ({\alpha}_l ( \prod_{k \in {\beta}_l} M_{R_{k}} v_x ) ),score(y|x) = v_y^T s \tag{3}
\end{equation*}

<small>whereas the score is then equivalent to entries in the vector s</small>

Learning for each query:

\begin{equation*}
max _{{\alpha}_l,{\beta}_l} \sum_{x,y} score(y|x) = max _{{\alpha}_l,{\beta}_l} \sum_{x,y} v_y^T (\sum_{l} ({\alpha}_l ( \prod_{k \in {\beta}_l} M_{R_{k}} v_x ) )) \tag{4}
\end{equation*}

<small>where ${x,y}$ are entity pairs that satisfy the query and ${\alpha}_l,{\beta}_l$ are to be learned.</small>

#### BUT

* we need to learn for each query a set of rules ${\beta}_l$ that imply it **and** the confidences ${\alpha}_l$
* so we want to learn the parameters and the structure ${\alpha}_l,{\beta}_l$

* in other words: Each parameter is associated with a particular rule -> discrete task

# 5. Primer: Recurrent Neuronal Networks and LSTMs

* RNNs are Neuronal Network that have a "memory". It captures information about what has been calculated so far. 
* In other words: They are networks with loops in them, allowing information to persist.
* ${h}_t = a(h_{t-1}, x) $, where ${h}_t$ and ${h}_{t-1}$ are rnn state at time step $t$ and $t-1$, $x$ is an input vector
* In theory RNNs can make use of information in arbitrarily long sequences, but in practice they are limited to looking back only a few steps. Here is what a typical RNN(folded/unfolded) looks like:

<div><img src="presentationFiles/RNN-unrolled.png" width=600 align="center"></div>

* Problem: Gap between the relevant information and the point where it is needed can become very large
* Solution: LSTMs

### Long Short-Term Memory

* introduced by Hochreiter & Schmidhuber (1997)
* are building blocks for RNNs
* capable of learning long-term dependencies

<div><img src="presentationFiles/LSTM3-chain.png" width=600 align="center"></div>


# 6. Primer: Attention Mechanism

* Focusing on part of a subset of the information the NN is given. 
* For example, an RNN can attend over the output of another RNN. At every time step, it focuses on different positions in the other RNN.

* One would like the attention to be differentiable, so that one can learn where to focus. To do this one uses a trick: one focus everywhere, just to different extents.

<div><img src="presentationFiles/attentionmechanism.png" width=600 align="center"></div>


# 7. Differentiable Learning part 2

* Task: Make equation (2): $\sum_{l} {\alpha}_l \prod_{k \in {\beta}_l} M_{R_{k}}$ differentiable

* How? By interchanging summation and product:

\begin{equation*}
\prod_{t=1}^T \sum_{k}^R {\alpha}_t^k M_{R_k} \tag{5}
\end{equation*}

where $T$ is the max length of Rules and $R$ is the number of relations in the knowledge base.

**BUT**
* At the cost of fixed rule length $T$

* So use recurrent formulation:

\begin{equation*}
\mathbf{u}_0 = \mathbf{v}_x \tag{6}
\end{equation*}

\begin{equation*}
\mathbf{u}_t = \sum_{k}^{R} \mathbf{a}_{t} \mathbf{M}_{R_{k}} (\sum_{\tau = 0}^{t-1} \mathbf{b}_{t}^\tau \mathbf{u}_{\tau}) \tag{7}
\end{equation*}

\begin{equation*}
\mathbf{u}_{T+1} = \sum_{\tau = 0}^{T} \mathbf{b}_{T+1}^{\tau} \mathbf{u}_{\tau} \tag{8}
\end{equation*}

where $u_t$ is _memory vector_, $b_t$ is _attention vector_ and $a_t$ _operator attention vector_ (attention vector for the operators)

* $u_t$ is _memory vector_: initialised with

\begin{equation*}
\mathbf{u}_0 = \mathbf{v}_x \tag{6}
\end{equation*}

* $h_t$ is controller a rnn : 

\begin{equation*}
\mathbf{h}_t = update(h_{t-1}, input) \tag{9}
\end{equation*}

* $a_t$ is _operator attention vector_:

\begin{equation*}
\mathbf{a}_t = softmax(\mathbf{W}{h_t} + \mathbf{b}) \tag{10}
\end{equation*}

* $b_t$ is _attention vector_:

\begin{equation*}
\mathbf{b}_t = softmax([\mathbf{h}_0 ... \mathbf{h}_{t+1}]^T * \mathbf{h}_t) \tag{11}
\end{equation*}

# Architecture
<div><img src="presentationFiles/Screenshot-2018-1-11 1702 08367 pdf.png" width=600 align="center"></div>

# Architecture with Formulas

<div><img src="presentationFiles/diagram.png" width=700 align="center"></div>

### Implemented Architecture
#### +Dataset generation

In [12]:
def _db_to_matrix_db(db, num_entity, num_relation):
    '''
    taken from data.py original source code _db_to_matrix_db
    
    
    each entry is a relation, list[0] is head tail parntership, 
                              list[1] whether fact exist and 
                              list[2] matrix size = num_entityXnum_entitiy  
                              
    so build a list[2] matrix set on list[0] a list[1] and then you have Mr
    
    '''
    matrix_db = {r: ([[0,0]], [0.], [num_entity, num_entity]) 
                 for r in xrange(num_relation)}
    for i, fact in enumerate(db):
        rel = fact[1]
        head = fact[0]
        tail = fact[2]
        value = 1.
        matrix_db[rel][0].append([head, tail])
        matrix_db[rel][1].append(value)
    return matrix_db

In [13]:
#generate facts, relations, and so on.

#add more entities
Entities = Entities + list(np.arange(len(Entities),21,1))
num_entity = len(Entities)
print('Entities: ',num_entity)
#add more facts
facts = [[Uber, HasOfficeInCity, USA],[NewYork, CityInCountry, USA], [Lyft, HasOfficeInCity, Paris],[Paris, CityInCountry, France]]
facts = facts + [[np.random.randint(0,21),np.random.randint(0,len(relations)),np.random.randint(0,21)] for x in xrange(500)]
num_facts = len(facts)
print('Facts', num_facts)
#realtions
num_relation = len(relations)
print('Relations', num_relation)

#shuffle
import random
random.shuffle(facts)
num_facts_quart = num_facts/4
num_facts_half = num_facts/2
matrix_db = _db_to_matrix_db(facts, num_entity, num_relation)

('Entities: ', 21)
('Facts', 504)
('Relations', 5)


In [14]:
### Train
train_set = facts
print(len(train_set))
matrix_db_train = _db_to_matrix_db(train_set, num_entity, num_relation)

504


### additional helper methods

In [15]:
def _clip_if_not_None(g, v, low, high):
    """ Clip not-None gradients to (low, high). """
    """ Gradient of T is None if T not connected to the objective. """
    if g is not None:
        return (tf.clip_by_value(g, low, high), v)
    else:
        return (g, v)

In [16]:
def _random_uniform_unit(r, c):
    """ Initialize random and unit row norm matrix of size (r, c). """
    bound = 6./ np.sqrt(c)
    init_matrix = np.random.uniform(-bound, bound, (r, c))
    init_matrix = np.array(map(lambda row: row / np.linalg.norm(row), init_matrix))
    return init_matrix

In [17]:
#inputs
#num_entity = len(E)
num_step = 3
num_query = len(relations)*2 #dont know why
query_embed_size = 128

with tf.name_scope('input'):
    
    with tf.name_scope('tails'):
        tails = tf.placeholder(tf.int32, [None])
    #tails = tf.constant(R)
    with tf.name_scope('heads'):
        heads = tf.placeholder(tf.int32, [None])
    #heads = tf.constant(E)
    
    targets = tf.one_hot(indices = heads, depth=num_entity)# = one hot encoding E_one_hot
    #print('Targets:', targets.eval())
    
    #TODO: what are those queries
    #ids for matrix
    with tf.name_scope('queries'):
        queries = tf.placeholder(tf.int32, [None, num_step])
    
    #creates a matrix
    query_embeddings_params = tf.Variable(_random_uniform_unit(num_query+1,query_embed_size), dtype=tf.float32)
    
    #What does this do: https://stackoverflow.com/questions/34870614/what-does-tf-nn-embedding-lookup-function-do
    #TODO: what are inputs ?
    rnn_input = tf.nn.embedding_lookup(query_embeddings_params, queries)
    
    #print('RNN input:', rnn_input.eval())

### Controller
LSTM modules

\begin{equation*}
\mathbf{h}_t = update(h_{t-1}, input) \tag{9}
\end{equation*}

In [18]:
#controller
rnn_state_size = 128
num_layer = 3
with tf.name_scope('rnn_controller'):

    #in: from inputs
    rnn_inputs = [tf.reshape(q, [-1, query_embed_size]) for q in tf.split(rnn_input, num_step, axis=1)]
    
    cell = tf.contrib.rnn.LSTMCell(rnn_state_size, state_is_tuple=True)

    multicells = tf.contrib.rnn.MultiRNNCell([cell] * num_layer, state_is_tuple=True)
    
    init_state = multicells.zero_state(tf.shape(tails)[0], tf.float32)
    
    rnn_outputs, final_state = tf.contrib.rnn.static_rnn(multicells, rnn_inputs, initial_state=init_state)

### Attention operator

\begin{equation*}
\mathbf{a}_t = softmax(\mathbf{W}{h_t} + \mathbf{b}) \tag{10}
\end{equation*}

In [19]:
#Attention operator

#TODO: this number what ?
num_operator = num_relation*2 

with tf.name_scope('attention_operator'):
    W = tf.Variable(np.random.randn(rnn_state_size, num_operator),dtype=tf.float32)
    
    b = tf.Variable(np.zeros((1,num_operator)), dtype=tf.float32)
    # attention_operators: a list of num_step lists,
    # each inner list has num_operator tensors,
    # each tensor of size (batch_size, 1).
    # Each tensor represents the attention over an operator. 
    attention_operators = [tf.split(tf.nn.softmax(tf.matmul(rnn_output, W) + b), num_operator, axis = 1) for rnn_output in rnn_outputs]

### Memories

\begin{equation*}
\mathbf{u}_0 = \mathbf{v}_x \tag{6}
\end{equation*}

In [20]:
with tf.name_scope('memories'):
    # memories: (will be) a tensor of size (batch_size, t+1, num_entity),
    # where t is the current step (zero indexed)
    # Then tensor represents currently populated memory cells
    memories = tf.expand_dims(tf.one_hot(indices = tails, depth = num_entity),1)
    #print('memories', memories.eval())

In [21]:
with tf.name_scope('database'):
    database = {r : tf.sparse_placeholder(dtype=tf.float32, name = "database_%d" %r) for r in xrange(num_operator/2)}


### (Attention) Memories

\begin{equation*}
\mathbf{b}_t = softmax([\mathbf{h}_0 ... \mathbf{h}_{t+1}]^T * \mathbf{h}_t) \tag{11}
\end{equation*}

and
\begin{equation*}
\sum_{\tau = 0}^{t-1} \mathbf{b}_{t}^\tau \mathbf{u}_{\tau} \tag{7 lastpart or 8}
\end{equation*}

if:
\begin{equation*}
\mathbf{u}_t = \sum_{k}^{R} \mathbf{a}_{t} \mathbf{M}_{R_{k}} \tag{7 frontpart}
\end{equation*}

else:
\begin{equation*}
\mathbf{u}_{T+1} = \sum_{\tau = 0}^{T} \mathbf{b}_{T+1}^{\tau} \mathbf{u}_{\tau} \tag{8}
\end{equation*}



In [22]:
norm = not False
thr = 1e-20

with tf.name_scope('attention_memories'):
    # attention_memories: (will be) a list of num_step tensors,
    # each of size (batch_size, t+1),
    # where t is the current step (zero indexed).
    # Each tensor represents the attention over currently populated memory cells. 
    attention_memories = []
        
    for t in xrange(num_step):
        #equation 11
        attention_memories.append(
            tf.nn.softmax(
            tf.squeeze(
            tf.matmul(
            tf.expand_dims(rnn_outputs[t],1),
            tf.stack(rnn_outputs[0:t+1], axis=2)),
                squeeze_dims = [1])))
        
        #equation 7 last part or 8
        memory_read = tf.squeeze(
            tf.matmul(
                tf.expand_dims(attention_memories[t], 1),
                                          memories), squeeze_dims = [1])
        
        #with tf.variable_scope('stop_criterion'):
        if t < num_step - 1:

            database_results = []
            memory_read = tf.transpose(memory_read)

            #equation 7 frontpart
            for r in xrange(num_operator/2):
                for op_matrix, op_attn in zip(
                    [database[r],
                    tf.sparse_transpose(database[r])],
                    [attention_operators[t][r],
                    attention_operators[t][r+num_operator/2]]):
                    #print('op_matrix', op_matrix)
                    #print('memory_nread', memory_read)
                    product = tf.sparse_tensor_dense_matmul(op_matrix, memory_read)
                    database_results.append(tf.transpose(product) * op_attn)
            
            added_database_results = tf.add_n(database_results)
            
            if norm:
                added_database_results /= tf.maximum(thr, tf.reduce_sum(added_database_results, axis=1, keep_dims=True))                
                
            memories = tf.concat([memories, tf.expand_dims(added_database_results, 1)], axis= 1)


        else:
            #eventually equation 8
            predictions = memory_read

### Optimization/Loss

In [23]:
with tf.name_scope('loss'):
    final_loss = - tf.reduce_sum(targets * tf.log(tf.maximum(predictions, thr)), 1)
    
accuracy = False
top_k = 6 #10
with tf.name_scope('accuracy'):
    if not accuracy: 
        in_top = tf.nn.in_top_k(predictions=predictions, targets=heads, k = top_k)
    else :
        _, indices = tf.nn.top_k(predictions, top_k, sorted=False)
        in_top = tf.equal(tf.squeeze(indices), heads)
    
learning_rate = 0.001
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    gvs = optimizer.compute_gradients(tf.reduce_mean(final_loss))
    capped_gvs = map(lambda(grad,var): _clip_if_not_None(grad, var, -5.,5.),gvs)
    optimizer_step = optimizer.apply_gradients(capped_gvs)
    
# create a summary for our cost and accuracy
tf.summary.scalar("final_loss", final_loss)
#tf.summary.scalar("in_top", in_top)

summary_op = tf.summary.merge_all()

In [24]:
init_op = tf.global_variables_initializer()

In [25]:
import os
max_epoch = 14
model_path='onlp_demo/output'
if not os.path.exists(model_path):
    os.makedirs(model_path)

saver = tf.train.Saver(max_to_keep=max_epoch)
saver = tf.train.Saver()

config = tf.ConfigProto()
config.gpu_options.allow_growth = False
config.log_device_placement = False
config.allow_soft_placement = True

### Training

In [26]:
def get_next_batch(sets, matrix_db, batch_size, start):
    
    end = start+batch_size
        
    hh, qq, tt = zip(*sets[start:end])
    
    next_start = end % len(sets)
    
    return qq, hh, tt, matrix_db, next_start

In [27]:
def train(sess, train_set, matrix_db):
    epoch = 0
    batch_size = 15
    num_train = len(train_set) 
    num_batch_train = num_train / batch_size + 1
    start = 0
    
    epoch_loss = []
    epoch_in_top = []  
    train_stats = []
    
    fetches = [final_loss, in_top, optimizer_step]
    #predict
    #fetches = [final_loss, in_top]
    while epoch < max_epoch:
        
        for batch in xrange(num_batch_train):
            
            (qq, hh ,tt, mdbr, start) = get_next_batch(train_set, matrix_db, batch_size, start)

            feed = {}

            feed[queries] = [[q] * (num_step - 1) + [num_query] for q in qq]
            feed[heads] = hh
            feed[tails] =  tt

            for r in xrange(num_operator/2):
                feed[database[r]] = tf.SparseTensorValue(*mdbr[r])
            
            #print('feed', feed)            
            f_update = sess.run(fetches, feed)
                     
            epoch_loss +=list(f_update[0])
            epoch_in_top += list(f_update[1])
        
        print('Epoch %d loss: %0.4f, In Top %0.4f.' %  (epoch, np.mean(epoch_loss),np.mean(epoch_in_top)))
            
        train_stats.append([epoch_loss, epoch_in_top])
        
        model = saver.save(sess, model_path,global_step=epoch)
        print('Model saved')
        
        all_test_in_top = [np.mean(x[1]) for x in train_stats]
        best_test_epoch = np.argmax(all_test_in_top)
        best_test = all_test_in_top[best_test_epoch]
        
        epoch += 1


In [28]:
# Launch the graph and run the ops.
with tf.Session(config = config) as sess:
    # Run the 'init' op
    sess.run(init_op)
    print('Session initialized ... ')
    
    writer = tf.summary.FileWriter(model_path, graph=tf.get_default_graph())
    print('Writer initialized ...')
    
    train(sess, train_set, matrix_db)
    print('Training finished ... ')
    writer.close()
    
    save_path = saver.save(sess, model_path+"/model.ckpt")
    print("Model saved in file: "+model_path)

    

Session initialized ... 
Writer initialized ...
Epoch 0 loss: 2.9988, In Top 0.5992.
Model saved
Epoch 1 loss: 2.6526, In Top 0.7535.
Model saved
Epoch 2 loss: 2.4895, In Top 0.8178.
Model saved
Epoch 3 loss: 2.4019, In Top 0.8507.
Model saved
Epoch 4 loss: 2.3424, In Top 0.8726.
Model saved
Epoch 5 loss: 2.2994, In Top 0.8866.
Model saved
Epoch 6 loss: 2.2674, In Top 0.8964.
Model saved
Epoch 7 loss: 2.2429, In Top 0.9042.
Model saved
Epoch 8 loss: 2.2235, In Top 0.9102.
Model saved
Epoch 9 loss: 2.2076, In Top 0.9151.
Model saved
Epoch 10 loss: 2.1945, In Top 0.9188.
Model saved
Epoch 11 loss: 2.1834, In Top 0.9220.
Model saved
Epoch 12 loss: 2.1739, In Top 0.9249.
Model saved
Epoch 13 loss: 2.1656, In Top 0.9271.
Model saved
Training finished ... 
Model saved in file: onlp_demo/output


# 8. Recovery of Logical Rules

* System has learned attention vectors $\{a_t, b_t\}$
* so we can write rules and the confidences based on them
* by applying the attention vectors to equation 7

\begin{equation*}
\mathbf{u}_t = \sum_{k}^{R} \mathbf{a}_{t} \mathbf{M}_{R_{k}} (\sum_{\tau = 0}^{t-1} \mathbf{b}_{t}^\tau \mathbf{u}_{\tau}) \tag{7}
\end{equation*}

and keeping track of the coefficients in front of each matrix $M_R$

* The procedure is the following:

**Input:** attention vectors $\{a_t \mid {t = 1,...,T}\}$ and $\{b_t \mid {t = 1,...,T+1}\}$  
**Notation:** Let $R_t = \{r_1,...,r_l\}$ be the set of partial rules at step $t$. Each rule $r_l$ is represented by a pair of $(\alpha, \beta)$ as described Equation 1, where $\alpha$ is the confidence and $\beta$ is ordered list of relation indexes.

**Initialize:** $R_0 = \{r_0\}$ where $r_0 =(1,())$.

**for** $\tau \leftarrow 1$ **to** $T+1$ **do:**

&nbsp;&nbsp;**Initialize:** $ \widehat{R_t} = \emptyset$, a placeholder for storing intermediate results.  
&nbsp;&nbsp;**for** $\tau \leftarrow 0$ **to** $t-1$ **do:**  
&nbsp;&nbsp;&nbsp;&nbsp;**for** $rule(\alpha,\beta)$ in $R_{\tau}$ **do**   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Update $\alpha \leftarrow \alpha*b_r^\tau$. Store the updated rule(\alpha,\beta) in $\widehat{R_t}$.   
&nbsp;&nbsp;&nbsp;&nbsp; **if** $t <= T$ **then**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Initialize** $R_t = \emptyset$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**for** $rule(\alpha,\beta)$ in $\widehat{R_t}$ **do**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**for** $k \leftarrow 1$ tp $R$ **do**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Update$\alpha \leftarrow \alpha*a_t^k$,$\beta \leftarrow \beta$ append $k$. Add the updated $rule(\alpha,\beta)$ to $R_t$.  
&nbsp;&nbsp;&nbsp;&nbsp;**else**
&nbsp;&nbsp;&nbsp;&nbsp;$R_t = \widehat{R_t}$  
**return** $R_{t+1}$

In [29]:
#gets attentions
def run_graph_attention(qq, hh ,tt, db_matrix, sess):
    mdb = db_matrix
    fetches = [attention_operators, attention_memories]
    feed = {}
    feed[queries] = [[q] * (num_step - 1) + [num_query] for q in qq]
    feed[heads] = hh
    feed[tails] =  tt

    
    for r in xrange(num_operator/2):
        feed[database[r]] = tf.SparseTensorValue(*mdb[r])
        

    fetched_update = sess.run(fetches, feed)

    return fetched_update[0], fetched_update[1]

In [30]:
def get_attention_given_queries(sess, query):
    qq = query
    hh = [0] * len(query)
    tt = [0] * len(query)
    mdb = {r: ([(0,0)], [0.], (num_entity, num_entity)) 
            for r in xrange(num_operator / 2)}
    fetched = run_graph_attention(qq, hh, tt, mdb, sess)
    return fetched[0], fetched[1]

In [31]:
#query_for_rules=list(set(zip(*train_set)[1])) = relations
import pickle

def get_attentions(sess):
    query_batches = [[i] for i in relations]
    print(query_batches)

    all_attention_operators= {}
    all_attention_memories = {}

    for query in query_batches:
        attention_operators, attention_memories = get_attention_given_queries(sess, query)
        
        for i in xrange(len(query)):
                all_attention_operators[query[i]] \
                                        = [[attn[i] 
                                        for attn in attn_step] 
                                        for attn_step in attention_operators]
                all_attention_memories[query[i]] = \
                                        [attn_step[i, :] 
                                        for attn_step in attention_memories]
    pickle.dump([all_attention_operators, all_attention_memories], 
                open(os.path.join(model_path, "attentions.pckl"), "w"))

    msg = "Attentions collected."
    print(msg)

    all_queries = reduce(lambda x,y: list(x) + list(y), query_batches, [])
    return all_attention_operators, all_attention_memories, all_queries

In [32]:
def list_rules(attn_ops, attn_mems, the):
    """
    Given attentions over operators and memories, 
    enumerate all rules and compute the weights for each.
    
    Args:
        attn_ops: a list of num_step vectors, 
                  each vector of length num_operator.
        attn_mems: a list of num_step vectors,
                   with length from 1 to num_step.
        the: early prune by keeping rules with weights > the
    
    Returns:
        a list of (rules, weight) tuples.
        rules is a list of operator ids. 
    
    """
    
    num_step = len(attn_ops)
    paths = {t+1: [] for t in xrange(num_step)}
    paths[0] = [([], 1.)]
    for t in xrange(num_step):
        for m, attn_mem in enumerate(attn_mems[t]):
            for p, w in paths[m]:
                paths[t+1].append((p, w * attn_mem))
        if t < num_step - 1:
            new_paths = []           
            for o, attn_op in enumerate(attn_ops[t]):
                for p, w in paths[t+1]:
                    if w * attn_op > the:
                        new_paths.append((p + [o], w * attn_op))
            paths[t+1] = new_paths
    this_the = min([the], max([w for (_, w) in paths[num_step]]))
    final_paths = filter(lambda x: x[1] >= this_the, paths[num_step])
    final_paths.sort(key=lambda x: x[1], reverse=True)
    
    return final_paths


In [33]:
parser = {"query":{}, "operator":{}}
relation_names = ['HasOfficeInCity', 'CityInCountry', 'HasOfficeInCountry', 'CountryIsInEurope', 'CountryIsInAsia']
entity_names = ['Uber/0', 'Lyft/1', 'USA/2', 'NewYork/3', 'France/4', 'Paris/5']
facts = [[Uber, HasOfficeInCity, USA],[NewYork, CityInCountry, USA], [Lyft, HasOfficeInCity, Paris],[Paris, CityInCountry, France]]


for value in xrange(num_relation):
    parser["query"][value] = relation_names[value]
    parser["query"][value + num_relation] = "inv_" + relation_names[value]

        
for query in xrange(num_relation):
    d = {}
    for k,v in enumerate(relation_names):
        
        d[k] = v
        d[k + num_relation] = "inv_" + v

            
    parser["operator"][query] = d
    parser["operator"][query + num_relation] = d

#parser

In [34]:
import sys
def get_rules(sess, parserm, b_print=True):
    all_attention_operators, all_attention_memories, query = get_attentions(sess)
    
    all_listed_rules = {}
    all_printed_rules = []
    for i, q in enumerate(query):
        if not False:
            if (i+1) % max(1, (len(query) / 5)) == 0:
                sys.stdout.write("%d/%d\t" % (i, len(query)))
                sys.stdout.flush()
        else: 
            # Tuple-ize in order to be used as dict keys
            q = tuple(q)
            
        #if q in attention_operators:
        #    if q in all_attention_memories:
        all_listed_rules[q] = list_rules(all_attention_operators[q], 
                                         all_attention_memories[q],
                                         0.01,)
        if b_print:
            all_printed_rules += print_rules(q, 
                                         all_listed_rules[q], parser)

    pickle.dump(all_listed_rules, 
                open(os.path.join(model_path, "rules.pckl"), "w"))
    if b_print:
        with open(os.path.join(model_path, "rules.txt"), "w") as f:
            for line in all_printed_rules:
                f.write(line + "\n")
    print("\nRules listed and printed.")

In [35]:
def print_rules(q_id, rules, parser):
    
    if len(rules) == 0:
        return []
    
    query = parser["query"][q_id]
    
    print(query)
    # assume rules are sorted from high to lows
    max_w = rules[0][1]
    # compute normalized weights also    
    rules = [[rule[0], rule[1], rule[1]/max_w] for rule in rules]

    printed_rules = [] 
    for rule, w, w_normalized in rules:
        if len(rule) == 0:
            printed_rules.append(
                "%0.3f (%0.3f)\t%s(B, A) <-- equal(B, A)" 
                % (w, w_normalized, query))
        else:
            lvars = [chr(i + 65) for i in xrange(1 + len(rule))]
            printed_rule = "%0.3f (%0.3f)\t%s(%c, %c) <-- " \
                            % (w, w_normalized, query, lvars[-1], lvars[0]) 
            for i, literal in enumerate(rule):
                if not False:
                    literal_name = parser["operator"][q_id][literal]
                else:
                    literal_name = parser["operator"][literal]
                printed_rule += "%s(%c, %c), " \
                                % (literal_name, lvars[i+1], lvars[i])
            printed_rules.append(printed_rule[0: -2])
    
    return printed_rules
    

In [36]:
# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, model_path+"/model.ckpt")
    print("Model restored.")
    # Check the values of the variables
    print('Getting rules')
    get_rules(sess, parser)

INFO:tensorflow:Restoring parameters from onlp_demo/output/model.ckpt
Model restored.
Getting rules
[[0], [1], [2], [3], [4]]
Attentions collected.
0/5	HasOfficeInCity
1/5	CityInCountry
2/5	HasOfficeInCountry
3/5	CountryIsInEurope
4/5	CountryIsInAsia

Rules listed and printed.


In [37]:
rulep = pickle.load(open(os.path.join(model_path, "rules.pckl"), "r"))
rulep

{0: [([0], array([ 0.44538364], dtype=float32)),
  ([0, 0], array([ 0.13930921], dtype=float32)),
  ([4, 0], array([ 0.08736341], dtype=float32)),
  ([2, 0], array([ 0.06028807], dtype=float32)),
  ([1, 0], array([ 0.04206241], dtype=float32)),
  ([7, 0], array([ 0.03499863], dtype=float32)),
  ([3, 0], array([ 0.03308333], dtype=float32)),
  ([9, 0], array([ 0.03084066], dtype=float32)),
  ([8, 0], array([ 0.03072744], dtype=float32)),
  ([5, 0], array([ 0.02913266], dtype=float32)),
  ([6, 0], array([ 0.02882993], dtype=float32))],
 1: [([1], array([ 0.4532178], dtype=float32)),
  ([1, 1], array([ 0.14183383], dtype=float32)),
  ([4, 1], array([ 0.07659588], dtype=float32)),
  ([2, 1], array([ 0.07201514], dtype=float32)),
  ([0, 1], array([ 0.04498353], dtype=float32)),
  ([7, 1], array([ 0.03413545], dtype=float32)),
  ([9, 1], array([ 0.03397993], dtype=float32)),
  ([5, 1], array([ 0.02965643], dtype=float32)),
  ([3, 1], array([ 0.02881895], dtype=float32)),
  ([6, 1], array([ 0

In [38]:
import csv
def open_rules_txt(filename = 'presentationFiles/rules.txt', N = 5):
    with open(filename, 'r') as myfile:
        head = [next(myfile) for x in xrange(N)]
    for each in head:
        print (each)

For a family example this looks like:

In [39]:
open_rules_txt(N = 5)

0.333 (1.000)	aunt(C, A) <-- mother(B, A), sister(C, B)

0.284 (0.853)	aunt(C, A) <-- inv_son(B, A), sister(C, B)

0.113 (0.339)	aunt(C, A) <-- father(B, A), sister(C, B)

0.111 (0.332)	aunt(C, A) <-- inv_daughter(B, A), sister(C, B)

0.062 (0.185)	aunt(C, A) <-- inv_nephew(B, A), sister(C, B)



In [40]:
open_rules_txt(filename=model_path+'/rules.txt',N = 5)

0.445 (1.000)	HasOfficeInCity(B, A) <-- HasOfficeInCity(B, A)

0.139 (0.313)	HasOfficeInCity(C, A) <-- HasOfficeInCity(B, A), HasOfficeInCity(C, B)

0.087 (0.196)	HasOfficeInCity(C, A) <-- CountryIsInAsia(B, A), HasOfficeInCity(C, B)

0.060 (0.135)	HasOfficeInCity(C, A) <-- HasOfficeInCountry(B, A), HasOfficeInCity(C, B)

0.042 (0.094)	HasOfficeInCity(C, A) <-- CityInCountry(B, A), HasOfficeInCity(C, B)



# 9. Experiments

* **Goal**: is to retrive a missing head from a query(head, tail) ,where query and tail are known. Example: $ HasOfficeInCountry(USA, Uber) $ is missing in knowledge base. So the goal is to reason over existing data tuples and retrieve $USA$ when presented with $ HasOfficeInCountry $ and $Uber$


* Parameters:
    * LSTM as RNN, hidden state size of 128
    * mini batch Adam, batch size of 64
    * initial learning rate: 0.001
    * max epoch: 10
    * log loss
    * early stopping
    * T = 2


* Datasets: WordNet, Freebase, Wn18, FB15k, FB15KSelected

* Statistics:

|                       |          | WN 18    |          | FB15K    |          | FB15KSelected |
|-----------------------|----------|----------|----------|----------|----------|---------------|
|                       | MRR      | Hits@10  | MRR      | Hits@10  | MRR      | Hits@10       |
| Neural Tensor Network | 0.53     | 66.1     | 0.25     | 41.4     | -        | -             |
| TransE                | 0.38     | 90.9     | 0.32     | 53.9     | -        | -             |
| DistMuLT              | 0.83     | 94.2     | 0.35     | 57.7     | 0.25     | 40.8          |
| Node+LinkFeat         | 0.94     | 94.3     | 0.82     | 87.0     | 0.23     | 34.7(42.8)    |
| Implicit ReasoNets    | -        | 95.3     | -        | **92.7** | -        | -             |
| NeuralLP              | **0.99** | **99.8** | **0.83** | 91.6     | **0.31** | **49.3**      |

# 10.  Evaluation Metrics
* Mean Reciprocal Rank: computes the averge of the reciprocal rank of the desired entities

* Hits@10 : computes the percentage of how many desired entities are ranked among top ten

# 11. Experiment: Question answering against knowledge base

* query is partially structured, posed in a natural language: "In which country does X have an office?"
* NLP can handle such queries, since the input to the RNN is a vector which can be encoded in any way.

* Dataset: Wikimovies

* Statistics:

| Model                    | Accuracy |
|--------------------------|----------|
| Memory Network           | 78.5     |
| QA System                | 93.5     |
| Key-Value Memory Network | 93.9     |
| Neural LP                | **94.6**     |

# 12. Summary

+ Tensorlog as a differentiable logic
+ Attenion and Memory Mechanisms in Neuronal Network architectures
+ NLP as an end-to-end differentiable method for learning structure and parameters in knowledge bases
+ NLP improves perfomance on benchmark datasets

# Tensorboard Graph

In [41]:
# TensorFlow Graph visualizer code
import numpy as np
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script src="//cdnjs.cloudflare.com/ajax/libs/polymer/0.3.3/platform.js"></script>
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:900px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [42]:

g=tf.get_default_graph()
# Simply call this to display the result. Unfortunately it doesn't save the output together with
# the Jupyter notebook, so we can only show a non-interactive image here.
show_graph(g)



# Resources:
* https://arxiv.org/abs/1702.08367
* http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/
* https://distill.pub/2016/augmented-rnns/
* http://colah.github.io/posts/2015-08-Understanding-LSTMs/
* https://machinelearningmastery.com/how-does-attention-work-in-encoder-decoder-recurrent-neural-networks/
* https://medium.com/@Ben_Obe/introduction-to-presenting-with-juypter-with-reveal-js-8e34a07081b2
* https://www.draw.io/
* https://blog.heuritech.com/2016/01/20/attention-mechanism/